In [1]:
import cv2
import numpy as np
import os
face_directory='./User_Faces/'

In [2]:
face_classifier=cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

# Get Training Data(Faces)

In [3]:
def detect_and_extract_faces(image):
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if(faces is ()):
        return None
    
    for (x,y,w,h) in faces:
        cropped_face=image[y:y+h,x:x+w] 
    return cropped_face

Initialize the device and Read the face frames

In [4]:
device=cv2.VideoCapture(0)
frame_count=0

while(1):
    _,frame=device.read()
    if(detect_and_extract_faces(frame) is not None):
        frame_count+=1
        face=cv2.resize(detect_and_extract_faces(frame),(200,200))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        if(frame_count==1):
            user_name=input('Enter User Name: \n')
            user_folder_name=face_directory+user_name
            os.mkdir(user_folder_name)
        
        image_name=user_folder_name+'/'+str(frame_count)+'.jpg'
        cv2.imwrite(image_name,face)
        
        cv2.putText(face,str(frame_count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Face Cropper',face)
        
    else:
        print('No Face exists! Please reposition yourself/increase lighting')
        pass
    if(cv2.waitKey(1)==13 or frame_count==100):
        break

device.release()
cv2.destroyAllWindows()
print('Face Samples Collected!')

No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
Enter User Name: 
rishab
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
No Face exists! Please reposition yourself/increase lighting
Face Samples Collected!


# Train the Model

Get the user Images

In [5]:
user_name=input('Who are you?? \n')
data_path=face_directory+user_name+'/'
files=os.listdir(data_path)

image_files=[]
for file in files:
    if(os.path.isfile(os.path.join(data_path,file))):
        image_files.append(file)

Who are you?? 
rishab


In [6]:
training_data=[]
labels=[]
for i,file in enumerate(image_files):
    
    image_path=data_path+image_files[i]
    print(image_path)
    images=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    training_data.append(np.array(images).astype('uint8'))
    labels.append(i)
    
labels=np.array(labels).astype('int32')

./User_Faces/rishab/1.jpg
./User_Faces/rishab/10.jpg
./User_Faces/rishab/100.jpg
./User_Faces/rishab/11.jpg
./User_Faces/rishab/12.jpg
./User_Faces/rishab/13.jpg
./User_Faces/rishab/14.jpg
./User_Faces/rishab/15.jpg
./User_Faces/rishab/16.jpg
./User_Faces/rishab/17.jpg
./User_Faces/rishab/18.jpg
./User_Faces/rishab/19.jpg
./User_Faces/rishab/2.jpg
./User_Faces/rishab/20.jpg
./User_Faces/rishab/21.jpg
./User_Faces/rishab/22.jpg
./User_Faces/rishab/23.jpg
./User_Faces/rishab/24.jpg
./User_Faces/rishab/25.jpg
./User_Faces/rishab/26.jpg
./User_Faces/rishab/27.jpg
./User_Faces/rishab/28.jpg
./User_Faces/rishab/29.jpg
./User_Faces/rishab/3.jpg
./User_Faces/rishab/30.jpg
./User_Faces/rishab/31.jpg
./User_Faces/rishab/32.jpg
./User_Faces/rishab/33.jpg
./User_Faces/rishab/34.jpg
./User_Faces/rishab/35.jpg
./User_Faces/rishab/36.jpg
./User_Faces/rishab/37.jpg
./User_Faces/rishab/38.jpg
./User_Faces/rishab/39.jpg
./User_Faces/rishab/4.jpg
./User_Faces/rishab/40.jpg
./User_Faces/rishab/41.jpg
./Us

Initialize and train the classifier

In [7]:
recognition_model=cv2.face.LBPHFaceRecognizer_create()
recognition_model.train(np.array(training_data),labels)
print('Model trained successfully!')

Model trained successfully!


# Run and Test

In [8]:
import cv2
import numpy as np
import os

face_directory='./User_Faces/'
face_classifier=cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

def extract_images_roi(image,size=0.5):
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if(faces is ()):
        return image,[]
    
    for (x,y,w,h) in faces:
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face=image[y:y+h,x:x+w]
        cropped_face=cv2.resize(cropped_face,(200,200))
    return image,cropped_face

In [10]:
device=cv2.VideoCapture(0)
frame_count=0

while(1):
    _,frame=device.read()
    image,face=extract_images_roi(frame)

    try:    
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        label,score=recognition_model.predict(face)
        
        if(score<500):
            confidence=int(100*(1-(score)/500))
            display=f'Confidence of given user is {confidence} \n'
            
        cv2.putText(image,display,(100,120),cv2.FONT_HERSHEY_COMPLEX,1,(255,125,150),2)
        
        if(confidence>75):
            cv2.putText(image,'Face Matched ! Unlocked.',(250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition',image)
        else:
            cv2.putText(image,'Face Not Matched ! Locked.',(250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition',image)

        
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

device.release()
cv2.destroyAllWindows()
print('Face Samples Collected!')

Face Samples Collected!
